In [1]:
import pandas as pd
from sqlalchemy import create_engine

## Create Dataframe

In [2]:
df = pd.read_csv("data.csv")

In [3]:
df.rename(columns={"Unnamed: 0": "id"}, inplace=True)

In [4]:
df.head()

,id,zipcode,landuse,bldgarea,numfloors,yearbuilt
0,0,10019.0,5.0,57805.0,3.0,1920.0
1,1,10019.0,7.0,105562.0,0.0,2002.0
2,2,10023.0,3.0,225552.0,24.0,1968.0
3,3,10019.0,6.0,36184.0,4.0,1946.0
4,4,11219.0,2.0,3600.0,2.0,1930.0


## Create Database file for Flask app

In [9]:
database_path = "../plutoDB.sqlite"

In [10]:
# Create Engine
# will create sqlite file if it doesn't already exist
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [11]:
# Push data to database
df.to_sql(name="pluto_table", con=conn, if_exists ="replace", index=False)

In [16]:
#Create primary key (doesn't work)
#conn.execute('ALTER TABLE `pluto_table` ADD CONSTRAINT PRIMARY KEY (`id`);')

In [27]:
# Query records to test that it works
data = pd.read_sql("SELECT * FROM pluto_table", conn)

In [28]:
data.head()

,id,zipcode,landuse,bldgarea,numfloors,yearbuilt
0,0,10019.0,5.0,57805.0,3.0,1920.0
1,1,10019.0,7.0,105562.0,0.0,2002.0
2,2,10023.0,3.0,225552.0,24.0,1968.0
3,3,10019.0,6.0,36184.0,4.0,1946.0
4,4,11219.0,2.0,3600.0,2.0,1930.0


In [31]:
value_type = "landuse"
zipcode = 10001

In [32]:
test = pd.read_sql(f"SELECT {value_type} FROM pluto_table WHERE zipcode = {zipcode}", conn)

In [33]:
# Code to generate simple list for javascript function
test.iloc[:, 0].tolist()

[5.0,
 4.0,
 5.0,
 4.0,
 3.0,
 8.0,
 5.0,
 5.0,
 5.0,
 6.0,
 5.0,
 11.0,
 5.0,
 5.0,
 5.0,
 5.0,
 10.0,
 4.0,
 4.0,
 10.0,
 5.0,
 5.0,
 6.0,
 11.0,
 8.0,
 9.0,
 8.0,
 5.0,
 6.0,
 11.0,
 5.0,
 5.0,
 2.0,
 10.0,
 10.0,
 6.0,
 6.0,
 3.0,
 5.0,
 3.0,
 5.0,
 6.0,
 11.0,
 5.0,
 5.0,
 5.0,
 2.0,
 11.0,
 5.0,
 5.0,
 3.0,
 4.0,
 5.0,
 5.0,
 8.0,
 3.0,
 11.0,
 3.0,
 4.0,
 4.0,
 4.0,
 4.0,
 11.0,
 5.0,
 4.0,
 5.0,
 4.0,
 5.0,
 11.0,
 8.0,
 5.0,
 8.0,
 7.0,
 8.0,
 5.0,
 5.0,
 7.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 3.0,
 5.0,
 8.0,
 5.0,
 4.0,
 4.0,
 5.0,
 5.0,
 3.0,
 5.0,
 5.0,
 6.0,
 4.0,
 7.0,
 11.0,
 4.0,
 4.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 2.0,
 3.0,
 5.0,
 8.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 3.0,
 2.0,
 10.0,
 4.0,
 6.0,
 6.0,
 2.0,
 5.0,
 8.0,
 5.0,
 5.0,
 4.0,
 5.0,
 2.0,
 4.0,
 5.0,
 9.0,
 6.0,
 2.0,
 5.0,
 8.0,
 5.0,
 7.0,
 5.0,
 11.0,
 9.0,
 5.0,
 5.0,
 5.0,
 6.0,
 4.0,
 5.0,
 5.0,
 5.0,
 2.0,
 5.0,
 5.0,
 4.0,
 5.0,
 7.0,
 8.0,
 5.0,
 4.0,
 2.0,
 5.0,
 2.0,
 4.0,
 3.0,
 2.0,
 11.0,


## Alternate method

In [ ]:
# This didn't end up working in Flask
# automap_base can't read the table name
# not sure why, but seems like there could be an issue with the primary key

In [ ]:
#https://datatofish.com/create-database-python-using-sqlite3/

In [5]:
import sqlite3
from sqlalchemy import inspect

In [6]:
database_path = "plutoDB.sqlite"

In [7]:
#conn = sqlite3.connect("plutoDB.db")
conn = sqlite3.connect(database_path)
c = conn.cursor()

In [8]:
# Create table
c.execute('''CREATE TABLE pluto_table
             ([id] INTEGER PRIMARY KEY, 
             [zipcode] integer, 
             [landuse] integer, 
             [numfloors] integer, 
             [bldgarea] integer, 
             [yearbuilt] integer)''')

conn.commit()

In [9]:
# Append, not replace, b/c you need to preserve the primary key
df.to_sql(database_path, con=conn, if_exists ="append", index=False)

In [10]:
# Create Engine
engine = create_engine(f"sqlite:///{database_path}")

In [11]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [12]:
# Collect the names of tables within the database
inspector.get_table_names()

['plutoDB.sqlite', 'pluto_table']

In [13]:
# Using the inspector to print the column names within the 'dow' table and its types
# columns will be a dictionary will many values, this prints the key/value pairs
columns = inspector.get_columns('pluto_table')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
zipcode INTEGER
landuse INTEGER
numfloors INTEGER
bldgarea INTEGER
yearbuilt INTEGER


In [14]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [15]:
# Declare a Base using `automap_base()`
Base = automap_base()

In [16]:
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [17]:
# Print all of the classes(tables) mapped to the Base
Base.classes.keys()

['pluto_table']

In [18]:
Pluto = Base.classes.pluto_table

In [19]:
session = Session(engine)

In [20]:
value_type = "landuse"
zip_code = 10019

In [21]:
data = session.query(Pluto[value_type]).filter(Pluto.zipcode == zip_code).all()

TypeError: 'DeclarativeMeta' object is not subscriptable